# App Building Tutorial

<a href = "#interactivity-and-callbacks">1. Interactivity and callbacks</a><br>
&nbsp;&nbsp;&nbsp;&nbsp;<a href = "#callbacks">1.1. Callbacks</a><br>
&nbsp;&nbsp;&nbsp;&nbsp;<a href = "#callbacks-widgets">1.2. Widgets</a><br>
&nbsp;&nbsp;&nbsp;&nbsp;<a href = "#callbacks-plots">1.3. Plots</a><br>
<a href = "#layout-templates">2. Layout templates</a><br>
&nbsp;&nbsp;&nbsp;&nbsp;<a href = "#app-layout">2.1. App layout</a><br>

In [2]:
import bqplot as bqp
from ipywidgets import AppLayout, Button, HBox, Image, IntSlider, Label, Layout, VBox
import numpy as np

# 1. Interactivity and callbacks<a name="interactivity-and-callbacks"></a>

In this section we're going to learn about callbacks and how BQuant uses them to make our applications interactive.

## 1.1. Callbacks<a name="callbacks"></a>

Before we understand exactly how we can use widgets and plots to make our applications interactive, we first need to learn about callbacks.

### What is a callback?

A callback is a piece of code we pass to other code to be executed at a given time. Just like we control a function's output by passing it different variables, we can also alter the function's behavior by passing it other functions.

In the example below, the `count_to_five` function accepts another arbitrary function, i.e. a callback. After `count_to_five` has printed the integers up to 5, it executes the callback.

In this case, `count_to_five` executes `example_callback` which just prints some text to the screen. However, we could instead pass any other arbitrary function. Callbacks therefore allow us to inject custom logic inside another function.

In [3]:
def count_to_five(func):
    for i in range(1, 6):
          print(i)
    # Execute the passed function by appending `()` to it
    func()

def example_callback():
    print('Callback triggered!')

count_to_five(example_callback)

1
2
3
4
5
Callback triggered!


## 1.2. Widgets<a name="callbacks-widgets"></a>

The various widgets in the `ipywidgets` library are the building blocks we use to create user interfaces for BQuant applications. 

### Widget callbacks and the `observe` method

BQuant helps us create dynamic applications by giving us the ability to bind a callback to a widget's property update. A widget's property is the main type of data it stores. For example, a slider with a value of 4 has a property of 4. A property update happens when that value is changed to something else, say 5. By binding the callback to that update, we can trigger some code every time a user interacts with a widget.

We can attach callbacks to widget property updates using the `observe` method. `observe` accepts a `handler`, i.e. the callback function, and a `names` keyword argument. `names` specifies which widget property we want to attach the callback to. In most cases we're interested in the `.value` attribute of the widget (what we referred to as the property of the widget above) so we can typically set `names='value'`.

Widget callbacks are automatically passed an argument, `change` in the example below, every time they are triggered. This is usually the state of the widget. It's important that you catch this argument by specifying an argument in your callback definition else the callback will fail!

Let's display the `change` object in a label to see what it is. Try moving the slider and see how the value of `change` responds.

In [15]:
slider = IntSlider()
slider_label = Label('')

def slider_callback(change):
    # Make sure to catch the default argument passed to the callback!
    slider_label.value = 'Widget state: ' + str(change)

slider.observe(slider_callback, names='value')
HBox([slider, slider_label])

It looks like `change` is a dictionary. This state dictionary tells us information about the property update that triggered the callback:
* The widget `'owner'` associated with the callback,
* The `'type'` of notification that triggered the callback,
* The `'name'` of the widget property,
* The previous property value of the widget, `'old'`,
* And the updated property value, `'new'`.

We can use the values in this dictionary to specify logic in our callback. One possibility would be to use the new value of the slider to specify a threshold for fetching some data.

It's important to reiterate that the above callback is arbitrary. We can specify any logic we want to inside a callback and then bind it to a widget property update.

### Buttons and the `on_click` method

A button doesn't store any state; it's only job is to handle mouse clicks. It therefore has a special `on_click` method that accepts a callback.

In [5]:
button = Button(description='Click me!')
button_label = Label('')

def on_click_callback(b):
    # The method gets passed the button itself as its only argument.
    button_label.value = str(b)
    
button.on_click(on_click_callback)
HBox([button, button_label])

### Callback errors

One of the most common widget gotchas has to do with errors inside callbacks.

In the code below, we are purposefully raising a `ValueError` inside a callback that we're then binding to a button.

What do you think will happen when you click the button? Try clicking it to find out.

In [6]:
broken_button = Button(description='Click me if you dare...')

def broken_callback(b):
    raise ValueError

# This should raise an error when we click the button
broken_button.on_click(broken_callback)
broken_button

Button(description='Click me if you dare...', style=ButtonStyle())

Nothing happened! You might have expected that you'd see an error message below the cell when you clicked the button.

The error message is being captured, just not here. If you open up the log console you'll see the error messages.

Watch the following gif for details on how to access to the console:

In [7]:
file = open('img/log_console.gif', 'rb')
image = file.read()
file.close()
Image(value=image, width=800)

Image(value=b'GIF89a\xbd\x03`\x02w\x00\x00!\xff\x0bNETSCAPE2.0\x03\x01\x00\x00\x00!\xf9\x04\x00\x0f\x00\x00\x0…

Visit the [official ipywidgets documentation](https://ipywidgets.readthedocs.io/en/latest/) website for more details.

## 1.3. Plots<a name="callbacks-plots"></a>

BQuant also lets us attach callbacks to `bqplot` plots. More specifically, we can bind callbacks to marks. The official `bqplot` documentation page describes marks as &ldquo;representations of the data&rdquo;&mdash;think clicking on a line in a lineplot or a point in a scatterplot.

Marks in `bqplot` have a selection of different functions for binding callbacks to specific actions. These are the equivalent of the `observe` method we used for widgets above:
* `on_hover`: Trigger callback whenever we hover over the mark.
* `on_background_click`: Trigger callback whenever we click on the background of the mark.
* `on_legend_click`: Trigger callback whenever we click on the mark's legend.
* `on_element_click`: Trigger callback whenever we click on a mark element.

In the example below, let's bind a simple print callback to all these different methods to see what get's passed to them.

In [8]:
x_data = np.arange(20)
y_data = np.random.randn(20)

x_scale = bqp.LinearScale()
y_scale = bqp.LinearScale()
scales = {'x': x_scale, 'y': y_scale}

scatter = bqp.Scatter(x=x_data,
                      y=y_data,
                      scales=scales,
                      colors=['dodgerblue'],
                      display_legend=True,
                      labels=['Blue'])

line = bqp.Lines(x=x_data,
                 y=y_data,
                 scales=scales,
                 colors=['dodgerblue'])

scatter2 = bqp.Scatter(x=x_data,
                       y=np.random.randn(20),
                       scales=scales,
                       colors=['red'],
                       display_legend=True,
                       labels=['Red'])

x_axis = bqp.Axis(scale=x_scale)
y_axis = bqp.Axis(scale=y_scale, orientation='vertical', tick_format='0.2f')

figure = bqp.Figure(marks=[scatter, scatter2, line],
                 axes=[x_axis, y_axis])

plot_label = Label('')

def print_event(self, target):
    plot_label.value = str(target)
    
scatter.on_hover(print_event)
scatter.on_background_click(print_event)

scatter2.on_element_click(print_event)
scatter2.on_legend_click(print_event)
line.on_element_click(print_event)

VBox([figure, plot_label])

Learn more by looking through interactice `bqplot` examples on <a href="https://mybinder.org/v2/gh/bloomberg/bqplot/stable?filepath=examples/Index.ipynb">binder</a>.

# 2. Layout templates<a name="layout-templates"></a>

In this section we're going to learn about the new `ipywidget` layout templates and why they make app building easier.

### Building apps the old way

We can use combinations of the `HBox` and `VBox` widgets from `ipywidgets` to create layouts for our applications. However, this sometimes leads to deeply nested lists of container widgets (think `HBox` inside `VBox` inside `VBox` etc.) for more complex applications. These structures can be difficult to update and repetitive to build.

`ipywidgets` recently released some new layout widgets that greatly simplify building common application layouts. We're going to focus on the `AppLayout` class but you can visit the [documentation](https://ipywidgets.readthedocs.io/en/latest/examples/Layout%20Templates.html) for details on other layouts like `TwoByTwoLayout` and `GridspecLayout`.

## 2.1. App layout<a name="app-layout"></a>

The `AppLayout` widget is a template for a common application. It includes a header, body, and footer. The body is made of a left, center, and right element. We can assign other widgets to each of these 5 elements.

In [9]:
def create_expanded_button(description, button_style):
    return Button(description=description,
                  button_style=button_style,
                  layout=Layout(height='auto', width='auto'))

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

AppLayout(header=header_button,
          left_sidebar=left_button,
          center=center_button,
          right_sidebar=right_button,
          footer=footer_button)

AppLayout(children=(Button(button_style='info', description='Left', layout=Layout(grid_area='left-sidebar', he…

Each element is optional. If we don't pass a value for one of the elements, `AppLayout` will smartly merge the remaining elements: 

In [10]:
AppLayout(header=header_button,
          center=center_button,
          right_sidebar=right_button)

AppLayout(children=(Button(button_style='success', description='Header', layout=Layout(grid_area='header', hei…

### Updating app components

What makes the new layout templates really useful is how we can leverage them to simplify updating applications.

Consider a simple dashboard that includes some buttons and sliders at the top and a plot below. Now you decide to add a second plot next to the first plot.

Previously, we would have to dig through the children of a collection of `HBox` and `VBox` widgets and then reassign the `.children` for one of these container widgets. 

With layout widgets, we can skip this process and directly assign a new widget, an `HBox` in the case below, that contains both plots to the `.center` attribute of our application.

In [11]:
plot_1 = create_expanded_button('Plot 1', 'warning')

app_layout = AppLayout(header=header_button,
                       center=plot_1,
                       right_sidebar=right_button)

plot_2 = create_expanded_button('Plot 2', 'warning')

plot_box = HBox([plot_1, plot_2], layout={'width': 'auto'})

# Set plot widths to 50% so they each take up half of the total width
plot_1.layout.width = '50%'
plot_2.layout.width = '50%'

app_layout.center = plot_box
app_layout

AppLayout(children=(Button(button_style='success', description='Header', layout=Layout(grid_area='header', hei…